<a href="https://colab.research.google.com/github/SourabhMagadum/Hindi-Sentiment-Analysis/blob/master/Hindi_sentiment_analysis%20best.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
# from tensorflow.python.keras import Sequential
from keras.layers import  Dense, Embedding, LSTM, Bidirectional,GRU
# from tensorflow.python.keras.layers import Dense, Embedding, LSTM, GRU 
#
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import Sequential
from tensorflow.keras.activations import sigmoid
#
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from keras import optimizers
from keras import layers
# from tensorflow.python.keras import layers
import tensorflow as tf

2024-05-04 12:52:59.110450: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-04 12:52:59.110581: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-04 12:52:59.246797: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import pandas as pd

# Assuming your CSV file is named 'HINDI_dataset.csv' in the same directory
df = pd.read_csv("/kaggle/input/mtgcn-lstm/DATA.csv", names=["review", "sentiment"],encoding='UTF-8')
df = df.dropna()
len(df['review'])
# Display the first few rows of the DataFrame
df = df.iloc[1:] 

In [3]:
df.head()

,review,sentiment
1,"నాటింగ్‌హామ్‌ : దక్షిణాఫ్రికా, ఆస్ట్రేలియా వంట...",neutral
2,ఆసీస్‌ మ్యాచ్‌లో శతకంతో ఫామ్‌లోకి వచ్చిన శిఖర్...,neg
3,దీంతో జుట్టులో సమీ కరణాలు మారనున్నాయి.,neutral
4,నాలుగులో కుదు రుకున్న కేఎల్‌ రాహుల్‌ ఓపెనింగ్‌...,neutral
5,"రాహుల్‌, రోహిత్‌తో పాటుగా బరిలో దిగితే నాలుగో ...",neg


In [4]:
print(df["sentiment"])
df["sentiment"] = [0 if i== "neg" else 1 if i=="pos" else 2 for i in df["sentiment"]]

1        neutral
2            neg
3        neutral
4        neutral
5            neg
          ...   
35138        pos
35139        pos
35140        neg
35141        pos
35142        pos
Name: sentiment, Length: 35142, dtype: object


In [5]:
df.head()

,review,sentiment
1,"నాటింగ్‌హామ్‌ : దక్షిణాఫ్రికా, ఆస్ట్రేలియా వంట...",2
2,ఆసీస్‌ మ్యాచ్‌లో శతకంతో ఫామ్‌లోకి వచ్చిన శిఖర్...,0
3,దీంతో జుట్టులో సమీ కరణాలు మారనున్నాయి.,2
4,నాలుగులో కుదు రుకున్న కేఎల్‌ రాహుల్‌ ఓపెనింగ్‌...,2
5,"రాహుల్‌, రోహిత్‌తో పాటుగా బరిలో దిగితే నాలుగో ...",0


In [6]:
df.shape

(35142, 2)

In [7]:
y = df["sentiment"]
X = df.drop(["sentiment"], axis = 1)

In [8]:
# 70/15/15 train/test/val split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=1)

X_train.head()

,review
911,"ఈ వన్డేలో స్మ_x005F_x007f_తి(90), మిథాలీ(63) క..."
130,ఈసారి సెర్బో బాస్నియన్‌ బాక్సర్‌ మిక్రో డ్రాజ్...
10393,ప్రపంచకప్‌లో భాగంగా ఈ నెల 30న ఇంగ్లండ్‌తో జరగన...
8516,శుక్రవారం మధ్యాహ్నం మూడు గంటలకు ముఖ్యమంత్రి నా...
10043,బర్మన్‌ రికార్డు


In [9]:
tk = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\]^_`{"}~\t\n')


tk.fit_on_texts(X['review'])

# + 1 for unknown token
vocab_size = len(tk.word_index) +1

X_train_seq = tk.texts_to_sequences(X_train['review'])
X_test_seq = tk.texts_to_sequences(X_test['review'])
X_val_seq = tk.texts_to_sequences(X_val['review'])
# Initializing max length of sentence to 20 words
max_length = 20

In [10]:
vocab_size

70969

In [11]:
# tk.word_index

In [12]:
print(vocab_size)


70969


In [13]:
# X_train_seq

In [14]:
#padding the sequences to make all the input sequences of the same length

X_train_seq_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_seq_pad = pad_sequences(X_test_seq, maxlen=max_length,padding='post')
X_val_seq_pad = pad_sequences(X_val_seq, maxlen=max_length,padding='post')


In [15]:
from tensorflow.keras.utils import to_categorical


In [16]:

le = LabelEncoder()
y_train_le = le.fit_transform(y_train)
y_test_le = le.transform(y_test)
y_val_le = le.transform(y_val)
y_train_oh = to_categorical(y_train_le)
y_test_oh = to_categorical(y_test_le)
y_val_oh = to_categorical(y_val_le)

In [17]:
y_train_le

array([1, 2, 2, ..., 2, 0, 0])

In [18]:
X_train_seq_pad

array([[    1,   677,  2585, ...,   811,     0,     0],
       [  461, 28594, 28595, ...,     0,     0,     0],
       [  413,   164,     1, ...,     0,     0,     0],
       ...,
       [   47,   156,    14, ...,     0,     0,     0],
       [  573,   193, 13848, ...,     0,     0,     0],
       [ 7762,  3528,   259, ...,     0,     0,     0]], dtype=int32)

In [19]:
from tensorflow.keras.callbacks import EarlyStopping

In [20]:
emb_dims = 256
model = Sequential()
model.add(Embedding(vocab_size, emb_dims,  embeddings_regularizer=tf.keras.regularizers.l2(0.0001)))
model.add(LSTM(units=16, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(3, activation='softmax'))
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
# Assuming X_train_seq_pad and y_train_oh are your training data (sequences and one-hot encoded labels)
history = model.fit(X_train_seq_pad, y_train_oh, epochs=4, batch_size=128,validation_data=(X_val_seq_pad, y_val_oh), shuffle=False)


Epoch 1/4
199/199 ━━━━━━━━━━━━━━━━━━━━ 13s 38ms/step - accuracy: 0.5210 - loss: 1.4269 - val_accuracy: 0.5418 - val_loss: 1.0028
Epoch 2/4
199/199 ━━━━━━━━━━━━━━━━━━━━ 7s 36ms/step - accuracy: 0.5463 - loss: 1.0012 - val_accuracy: 0.5909 - val_loss: 0.9604
Epoch 3/4
199/199 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.6336 - loss: 0.9296 - val_accuracy: 0.6445 - val_loss: 0.9712
Epoch 4/4
199/199 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.7472 - loss: 0.7911 - val_accuracy: 0.6445 - val_loss: 1.0689


In [22]:
from sklearn.metrics import f1_score, precision_score, recall_score
y_pred = model.predict(X_test_seq_pad)
y_pred_classes = np.argmax(y_pred, axis=1)  # Assuming y_pred is one-hot encoded
y_test_classes = le.inverse_transform(y_test_le)  # Convert back to original classes


165/165 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step


In [23]:
f1 = f1_score(y_test_classes, y_pred_classes, average='macro')  # Macro averaging for imbalanced data (optional)
precision = precision_score(y_test_classes, y_pred_classes, average='macro')  # Macro averaging
recall = recall_score(y_test_classes, y_pred_classes, average='macro')  # Macro averaging

print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")


F1 Score: 0.5879
Precision: 0.6082
Recall: 0.5802


In [24]:
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 20, 256)        │    18,168,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 16)             │        17,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 3)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 54,556,763 (208.12 MB)

 Trainable params: 18,185,587 (69.37 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 36,371,176 (138.75 MB)

In [25]:
results = model.evaluate(X_test_seq_pad, y_test_oh)
print('/n')
print('Test accuracy of word embeddings model: {0:.2f}%'.format(results[1]*100))

165/165 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.6361 - loss: 1.0623
/n
Test accuracy of word embeddings model: 63.94%


In [26]:
results = model.evaluate(X_val_seq_pad, y_val_oh)
print('/n')
print('Validation accuracy of word embeddings model: {0:.2f}%'.format(results[1]*100))

141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6563 - loss: 1.0528
/n
Validation accuracy of word embeddings model: 64.45%
